# 04 高度な信号処理
## スペクトル分析と統計的手法

このノートブックでは、**ウェーブレット変換**や**スペクトル分析**などの高度な信号処理手法を学び、信号のより深い理解を目指します。

**学習目標:**
- ウェーブレット変換による時系列-周波数分析
- スペクトル分析の高度な手法
- 信号の統計的特性分析
- フレームデータセットの活用

**前提条件:**
- 03_signal_processing_basics.ipynb を完了していること

## 🎯 なぜ高度な手法が必要か

### 基本的な信号処理の限界

**FFTや基本フィルタ**には限界があります：

1. **時間分解能 vs 周波数分解能**のトレードオフ
2. **定常信号**の仮定（実際の信号は変化する）
3. **局所的な特徴**を捉えにくい
4. **非線形特性**を扱えない

### 高度な手法で何ができるか

- **ウェーブレット**: 時間と周波数の両方で局所的な分析
- **スペクトル推定**: より正確な周波数特性の推定
- **統計分析**: 信号の確率分布や相関の分析
- **非線形処理**: 複雑な信号モデルの適用

### Wandasの高度な機能

- **統合されたAPI**: 基本処理から高度処理まで統一インターフェース
- **効率的な実装**: NumPy/Daskによる高速処理
- **拡張性**: カスタム処理の容易な追加
- **可視化**: 複雑な結果も直感的に理解できる

## 🌊 ウェーブレット変換

### FFT vs ウェーブレット

**FFT（高速フーリエ変換）**:
- **全体の周波数特性**を一度に分析
- **時間情報が失われる**
- **定常信号**に適する

**ウェーブレット変換**:
- **時間と周波数の両方**で局所的な分析
- **時間変化する周波数特性**を捉えられる
- **非定常信号**に適する
- **スケール**の概念（粗い特徴から細かい特徴まで）

In [ ]:
# 必要なライブラリをインポート
import wandas as wd
import numpy as np
import matplotlib.pyplot as plt

# インタラクティブプロット設定
%matplotlib widget
plt.rcParams['figure.figsize'] = (12, 6)

print(f"Wandas: {wd.__version__}")
print("✅ 準備完了")

In [ ]:
# ウェーブレット分析に適した信号を作成
np.random.seed(42)
sampling_rate = 1000  # 1kHz
duration = 4.0
time = np.linspace(0, duration, int(duration * sampling_rate))

# 時間変化する周波数特性を持つ信号
wavelet_signal = np.zeros_like(time)

# 0-1秒: 低周波（10Hz）
mask1 = (time >= 0) & (time < 1)
wavelet_signal[mask1] = 2.0 * np.sin(2 * np.pi * 10 * time[mask1])

# 1-2秒: 中周波（50Hz）
mask2 = (time >= 1) & (time < 2)
wavelet_signal[mask2] = 1.5 * np.sin(2 * np.pi * 50 * time[mask2])

# 2-3秒: 高周波（150Hz）
mask3 = (time >= 2) & (time < 3)
wavelet_signal[mask3] = 1.0 * np.sin(2 * np.pi * 150 * time[mask3])

# 3-4秒: 複合周波数
mask4 = (time >= 3) & (time < 4)
wavelet_signal[mask4] = (
    0.8 * np.sin(2 * np.pi * 80 * time[mask4]) +
    0.6 * np.sin(2 * np.pi * 120 * time[mask4])
)

# ノイズを追加
wavelet_signal += 0.1 * np.random.randn(len(time))

# ChannelFrame作成
wavelet_data = wd.from_numpy(
    data=wavelet_signal.reshape(1, -1),
    sampling_rate=sampling_rate,
    ch_labels=['Wavelet Demo Signal']
)

print(f"ウェーブレット用信号作成: {wavelet_data.shape}, {wavelet_data.sampling_rate} Hz")

In [ ]:
# 時間領域での信号表示
wavelet_data.plot(title="Time-Varying Frequency Signal",
                  xlabel="Time [s]", ylabel="Amplitude")
plt.show()

In [ ]:
# スペクトログラム（比較用）
spec = wavelet_data.spectrogram(n_fft=256, hop_length=128)
spec.plot(title="Spectrogram (FFT-based)", ylabel="Frequency [Hz]", xlabel="Time [s]")
plt.show()

In [ ]:
# ウェーブレット変換（Continuous Wavelet Transform）
cwt_result = wavelet_data.cwt(
    wavelet='morlet',    # ウェーブレット関数
    scales=np.arange(1, 128),  # スケール（周波数に対応）
    sampling_period=1/sampling_rate
)

print("🌊 連続ウェーブレット変換:")
print(f"  ウェーブレット: Morlet")
print(f"  スケール数: {len(cwt_result.scales)}")
print(f"  時間サンプル数: {cwt_result.data.shape[1]}")
print(f"  周波数範囲: {cwt_result.freqs.min():.1f} - {cwt_result.freqs.max():.1f} Hz")

# ウェーブレット変換結果を表示
cwt_result.plot(title="Continuous Wavelet Transform",
                ylabel="Frequency [Hz]", xlabel="Time [s]")
plt.show()

**🌊 ウェーブレット変換の利点**

- **時間分解能**: 各時刻での周波数特性が明確
- **周波数分解能**: 低周波では粗く、高周波では細かい分析
- **遷移の検出**: 周波数変化のタイミングが正確
- **スケール可変**: 異なる解像度での分析が可能

スペクトログラムと比較して、**より鮮明な時間-周波数分解**が得られます。

## 📊 高度なスペクトル分析

### 古典的なスペクトル推定 vs 現代的な手法

**古典的な手法（Periodogram）**:
- **単純なFFTの振幅二乗**
- **バイアスと分散が大きい**
- **スペクトルの漏れ**が発生しやすい

**現代的な手法**:
- **Welch法**: 平均化による分散低減
- **Burg法**: ARモデルによる高解像度推定
- **MUSIC法**: 固有値分解による高精度推定
- **多重ウィンドウ法**: バイアス-分散のトレードオフ最適化

In [ ]:
# スペクトル推定の比較用信号（低SNR）
np.random.seed(123)
noisy_signal = (
    0.5 * np.sin(2 * np.pi * 100 * time) +  # 弱い信号
    0.3 * np.sin(2 * np.pi * 150 * time) +  # もっと弱い信号
    2.0 * np.random.randn(len(time))       # 強いノイズ
)

noisy_data = wd.from_numpy(
    data=noisy_signal.reshape(1, -1),
    sampling_rate=sampling_rate,
    ch_labels=['Low SNR Signal']
)

print(f"低SNR信号作成: SNR ≈ {10 * np.log10((0.5**2 + 0.3**2) / np.var(2.0 * np.random.randn(1000))):.1f} dB")

In [ ]:
# さまざまなスペクトル推定手法の比較
print("📊 スペクトル推定手法の比較:")

# 1. Periodogram（古典的）
periodogram = noisy_data.periodogram()

# 2. Welch法（平均化）
welch = noisy_data.welch(nperseg=1024, noverlap=512)

# 3. Burg法（ARモデル）
burg = noisy_data.burg(order=20)

print(f"Periodogram: {periodogram.shape}")
print(f"Welch: {welch.shape}")
print(f"Burg: {burg.shape}")

# 結果の比較プロット
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 12))

periodogram.plot(ax=ax1, title="Periodogram")
ax1.axvline(100, color='red', linestyle='--', alpha=0.7, label='True 100Hz')
ax1.axvline(150, color='green', linestyle='--', alpha=0.7, label='True 150Hz')
ax1.legend()

welch.plot(ax=ax2, title="Welch Method")
ax2.axvline(100, color='red', linestyle='--', alpha=0.7)
ax2.axvline(150, color='green', linestyle='--', alpha=0.7)

burg.plot(ax=ax3, title="Burg Method (AR)")
ax3.axvline(100, color='red', linestyle='--', alpha=0.7)
ax3.axvline(150, color='green', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

**📈 スペクトル推定手法の比較**

- **Periodogram**: ノイズが多く、ピークが不明瞭
- **Welch法**: 平均化によりノイズが低減、ピークがより明確
- **Burg法**: 高解像度でピークが鮮明、ARモデルの特性

低SNR信号では**Welch法やBurg法**の方が優れた性能を発揮します。

## 📈 信号の統計的特性分析

### 確率分布と統計量

信号の**統計的特性**を分析することで：
- **分布形状**: 正規分布か、非対称か
- **外れ値**: 異常値の検出
- **相関性**: 信号間の関係
- **定常性**: 統計的性質の時間変化

In [ ]:
# さまざまな分布の信号を作成
np.random.seed(456)

# 1. 正規分布信号
normal_signal = np.random.normal(0, 1, len(time))

# 2. 指数分布信号（片側分布）
exp_signal = np.random.exponential(1, len(time)) - 1  # 平均0に調整

# 3. ラプラス分布信号（尖った分布）
laplace_signal = np.random.laplace(0, 0.5, len(time))

# 4. 混合信号（正規 + インパルス）
mixed_signal = np.random.normal(0, 0.5, len(time))
impulse_indices = np.random.choice(len(time), size=50, replace=False)
mixed_signal[impulse_indices] += np.random.choice([-3, 3], size=50)

# ChannelFrame作成
stat_signals = wd.from_numpy(
    data=np.vstack([normal_signal, exp_signal, laplace_signal, mixed_signal]),
    sampling_rate=sampling_rate,
    ch_labels=['Normal', 'Exponential', 'Laplace', 'Mixed']
)

print(f"統計分析用信号作成: {stat_signals.shape}")

In [ ]:
# 統計量の計算
print("📊 信号の統計的特性:")

stats = {
    'mean': stat_signals.mean,
    'std': stat_signals.std,
    'skewness': stat_signals.skewness,  # 歪度
    'kurtosis': stat_signals.kurtosis,  # 尖度
    'rms': stat_signals.rms,
    'crest_factor': stat_signals.crest_factor  # クレストファクター
}

for stat_name, values in stats.items():
    print(f"\n{stat_name.upper()}:")
    for ch_name, value in zip(stat_signals.labels, values):
        print(f"  {ch_name}: {value:.3f}")

# ヒストグラムと分布の比較
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i, ch_name in enumerate(stat_signals.labels):
    ch_data = stat_signals[ch_name]

    # 時間波形
    axes[0, i].plot(ch_data.time[:1000], ch_data.data[0, :1000], 'b-', alpha=0.7)
    axes[0, i].set_title(f'{ch_name} - Time Series')
    axes[0, i].set_xlabel('Time [s]')
    axes[0, i].set_ylabel('Amplitude')
    axes[0, i].grid(True, alpha=0.3)

    # ヒストグラム
    axes[1, i].hist(ch_data.data.flatten(), bins=50, alpha=0.7, density=True)
    axes[1, i].set_title(f'{ch_name} - Distribution')
    axes[1, i].set_xlabel('Value')
    axes[1, i].set_ylabel('Density')
    axes[1, i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**📊 統計量の解釈**

- **Normal**: 歪度≈0, 尖度≈3（理想的な正規分布）
- **Exponential**: 歪度>0（右に裾が長い）
- **Laplace**: 尖度>3（中央が尖った分布）
- **Mixed**: クレストファクター大（インパルス性が高い）

これらの統計量は**信号の特性を定量的に評価**するのに役立ちます。

## 🔗 フレームデータセットの活用

### 複数の信号フレームをまとめて扱う

**FrameDataset**は複数のChannelFrameをまとめて管理するクラス：
- **バッチ処理**: 複数の信号に同じ処理を適用
- **グループ化**: 条件による信号の分類
- **統計分析**: データセット全体の特性分析
- **効率性**: 大規模データのメモリ効率的処理

In [ ]:
# 複数の信号からFrameDatasetを作成
np.random.seed(789)

# 異なる条件の信号を生成
conditions = ['Normal', 'High_Freq', 'Low_SNR', 'Transient']
frames = []

for condition in conditions:
    if condition == 'Normal':
        signal = np.sin(2 * np.pi * 50 * time) + 0.1 * np.random.randn(len(time))
    elif condition == 'High_Freq':
        signal = np.sin(2 * np.pi * 200 * time) + 0.1 * np.random.randn(len(time))
    elif condition == 'Low_SNR':
        signal = 0.1 * np.sin(2 * np.pi * 50 * time) + np.random.randn(len(time))
    elif condition == 'Transient':
        signal = np.zeros_like(time)
        start_idx = len(time) // 2
        signal[start_idx:start_idx+500] = np.sin(2 * np.pi * 100 * time[start_idx:start_idx+500])

    frame = wd.from_numpy(
        data=signal.reshape(1, -1),
        sampling_rate=sampling_rate,
        ch_labels=[f'{condition}_signal']
    )

    # メタデータを追加
    frame.metadata = {'condition': condition, 'snr': 10 * np.log10(np.var(signal) / 0.1**2)}
    frames.append(frame)

# FrameDataset作成
dataset = wd.FrameDataset(frames, labels=conditions)

print(f"FrameDataset作成: {len(dataset)} フレーム")
print(f"条件: {dataset.labels}")
for i, frame in enumerate(dataset.frames):
    print(f"  {dataset.labels[i]}: {frame.shape}, SNR: {frame.metadata['snr']:.1f} dB")

In [ ]:
# データセット全体の統計分析
print("📈 データセット全体の分析:")

# 全フレームのRMSを計算
rms_values = [frame.rms[0] for frame in dataset.frames]
print(f"RMS値: {dict(zip(dataset.labels, rms_values))}")

# 条件ごとのスペクトル分析
spectra = [frame.fft() for frame in dataset.frames]

# 比較プロット
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for i, (spectrum, label) in enumerate(zip(spectra, dataset.labels)):
    spectrum.plot(ax=axes[i], title=f'{label} Spectrum')
    axes[i].set_ylim(0, 0.5)  # Y軸を統一

plt.tight_layout()
plt.show()

In [ ]:
# データセット全体のバッチ処理
print("🔄 データセット全体のバッチ処理:")

# 全フレームに同じ処理を適用
processed_dataset = dataset.apply(
    lambda frame: (
        frame
        .high_pass_filter(cutoff=5)   # DC除去
        .low_pass_filter(cutoff=300)  # 高周波ノイズ除去
        .normalize()                   # 正規化
    )
)

print(f"バッチ処理完了: {len(processed_dataset)} フレーム")

# 処理結果の比較
fig, axes = plt.subplots(2, 4, figsize=(18, 8))

for i, (orig, proc, label) in enumerate(zip(dataset.frames, processed_dataset.frames, dataset.labels)):
    # 元信号
    orig.plot(ax=axes[0, i], title=f'{label} - Original')

    # 処理後
    proc.plot(ax=axes[1, i], title=f'{label} - Processed')

plt.tight_layout()
plt.show()

## 🚀 統合された高度なワークフロー

### 現実的な信号分析パイプライン

高度な手法を組み合わせた**完全な分析ワークフロー**を体験しましょう。

In [ ]:
# 複雑な現実的信号を作成
np.random.seed(999)
complex_duration = 10.0
complex_sr = 2000
complex_time = np.linspace(0, complex_duration, int(complex_duration * complex_sr))

# 複数の成分を持つ複雑な信号
complex_signal = (
    # 定常成分
    1.0 * np.sin(2 * np.pi * 60 * complex_time) +   # 基本周波数
    0.5 * np.sin(2 * np.pi * 120 * complex_time) +  # 2倍周波数
    0.3 * np.sin(2 * np.pi * 180 * complex_time) +  # 3倍周波数

    # 非定常成分（時間変化）
    0.8 * np.sin(2 * np.pi * 100 * complex_time) * np.exp(-complex_time/5) +  # 減衰

    # ノイズ
    0.2 * np.random.randn(len(complex_time)) +  # 白色ノイズ

    # インパルス（異常）
    0.0
)

# ランダムなインパルスを追加
impulse_times = np.random.choice(len(complex_time), size=20, replace=False)
complex_signal[impulse_times] += np.random.choice([-2, 2], size=20) * np.random.exponential(1, 20)

# ChannelFrame作成
complex_data = wd.from_numpy(
    data=complex_signal.reshape(1, -1),
    sampling_rate=complex_sr,
    ch_labels=['Complex Vibration']
)

print(f"複雑信号作成: {complex_data.shape}, {complex_data.sampling_rate} Hz")

In [ ]:
# 高度な信号分析ワークフロー
print("🚀 高度な信号分析ワークフロー:")

# 1. 基本統計
print("1. 📊 基本統計")
basic_stats = {
    'RMS': complex_data.rms[0],
    'Peak': complex_data.abs().max[0],
    'Crest Factor': complex_data.crest_factor[0],
    'Skewness': complex_data.skewness[0],
    'Kurtosis': complex_data.kurtosis[0]
}
for key, value in basic_stats.items():
    print(f"   {key}: {value:.3f}")

# 2. スペクトル分析
print("\n2. 🔄 スペクトル分析")
spectrum = complex_data.welch(nperseg=2048, noverlap=1024)
peak_indices = np.argsort(np.abs(spectrum.data[0]))[-5:][::-1]
peak_freqs = spectrum.freqs[peak_indices]
print(f"   主な周波数成分: {peak_freqs} Hz")

# 3. ウェーブレット分析
print("\n3. 🌊 ウェーブレット分析")
cwt = complex_data.cwt(scales=np.arange(1, 64), wavelet='morlet')
print(f"   ウェーブレット変換完了: {cwt.shape}")

# 4. 異常検出（統計的手法）
print("\n4. 🔍 異常検出")
# 移動RMSで異常を検出
window_size = int(0.1 * complex_sr)  # 100ms窓
moving_rms = np.sqrt(np.convolve(complex_data.data[0]**2,
                                np.ones(window_size)/window_size,
                                mode='same'))
anomaly_threshold = np.mean(moving_rms) + 3 * np.std(moving_rms)
anomalies = moving_rms > anomaly_threshold
print(f"   検出された異常点: {np.sum(anomalies)} サンプル")

# 5. 総合可視化
print("\n5. 📈 総合可視化")
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 時間波形 + 異常点
complex_data.plot(ax=ax1, title="Time Series with Anomalies")
anomaly_times = complex_time[anomalies]
ax1.scatter(anomaly_times, complex_data.data[0, anomalies],
           color='red', s=50, alpha=0.7, label='Anomalies')
ax1.legend()

# スペクトル
spectrum.plot(ax=ax2, title="Power Spectral Density")
for freq in peak_freqs[:3]:
    ax2.axvline(freq, color='red', linestyle='--', alpha=0.7)

# ウェーブレット
cwt.plot(ax=ax3, title="Wavelet Transform")

# 移動RMS
ax4.plot(complex_time, moving_rms, 'b-', alpha=0.7, label='Moving RMS')
ax4.axhline(anomaly_threshold, color='red', linestyle='--', alpha=0.7, label='Threshold')
ax4.fill_between(complex_time, 0, moving_rms, where=anomalies,
                color='red', alpha=0.3, label='Anomalies')
ax4.set_title("Anomaly Detection (Moving RMS)")
ax4.set_xlabel("Time [s]")
ax4.set_ylabel("RMS")
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ 高度な信号分析ワークフロー完了！")

## 🛠️ トラブルシューティング

### 高度な手法のよくある問題

#### 1. ウェーブレット変換の計算コスト
- **原因**: スケール数と時間サンプル数の積が大きい
- **対策**: スケール数を減らす、時間解像度を下げる
- **代替**: スペクトログラムを使用

#### 2. スペクトル推定の解釈
- **バイアス vs 分散**: Welch法はバイアス小、Burg法は分散小
- **周波数分解能**: セグメント長で決まる
- **スペクトルの漏れ**: 窓関数で軽減

#### 3. 統計量の頑健性
- **外れ値の影響**: 平均より中央値、分散よりMADを使用
- **分布の仮定**: 正規分布を仮定せずノンパラメトリック手法を検討

#### 4. FrameDatasetのメモリ使用
- **大規模データ**: 遅延評価（Dask）を使用
- **ストリーミング**: 一度に全てをメモリに載せない
- **圧縮**: 必要に応じてデータを圧縮

## 📚 次のステップ

高度な信号処理手法をマスターしました！

**次のノートブック**: [05_visualization_and_insights.ipynb](05_visualization_and_insights.ipynb)

ここでは、**高度な可視化手法**と**データからの洞察抽出**について紹介します。

### 🎯 これまでに学んだこと
- ✅ ウェーブレット変換による時系列-周波数分析
- ✅ 高度なスペクトル推定（Welch法、Burg法）
- ✅ 信号の統計的特性分析（歪度、尖度、クレストファクター）
- ✅ FrameDatasetによる複数信号の一括処理
- ✅ 統合された高度な信号分析ワークフロー

### 🚀 次の学習目標
- インタラクティブな可視化
- 多次元データの可視化
- 統計的可視化と分布分析
- 機械学習との統合
- レポート生成と共有

---

**高度な信号処理を身につけました。データから洞察を抽出する世界へ！** 🔬